# ***NOTEBOOK 4: Filtrado Colaborativo***

## **Carga de los datos**

In [ ]:
!pip install google-cloud-storage

In [ ]:
!pip install pyspark

  Using cached py4j-0.10.9.5-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'METADATA'
Consider using the `--user` option or check the permissions.



In [ ]:
!pip install sentence-transformers

  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cus

In [ ]:
!pip install scikit-surprise

In [ ]:
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy
import scipy.sparse as sp

*   **Dataset: Recipes**

In [ ]:
from google.cloud import storage

# Configura el nombre de tu bucket y archivo
bucket_name = 'mi-bucket-tfg'
file_path = 'datos/df_recipe_aspects.pkl'
local_path = '/tmp/df_recipe_aspects.pkl'

# Conectar a Google Cloud Storage
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename(local_path)

# Cargar el archivo Pickle en un DataFrame
df_recipes = pd.read_pickle(local_path)
df_recipes.head()

,name,description,servings,recipe_id,recipe_grams,recipe_minutes,ingredients_list,ingredients_and_quantities,instructions,keywords,...,fiber_100g,sugar_100g,protein_100g,time_group,avg_rating,review_count,n_ingredients,nutriscore,highlighted_aspects,criticized_aspects
0,Bourbon Chicken,I searched and finally found this recipe on th...,4,45809.0,330.0,35.0,"[boneless chicken breasts, olive oil, garlic c...","[2 lbs boneless chicken breasts, cut into bite...",[1. Editor's Note: Named Bourbon Chicken beca...,"[chicken, low-carb, dinner, chinese, 60-minute...",...,0.09,6.52,15.18,30-40,4.31,2892.0,11,C,"[great thing, best seller, aabout thick perfec...",[]
1,Creamy Cajun Chicken Pasta,N'awlin's style of chicken with an updated Alf...,2,39087.0,373.0,25.0,"[boneless skinless chicken breast halves, ling...","[2 boneless skinless chicken breast halves, cu...",[1. Place chicken and Cajun seasoning in a bow...,"[cajun, southern, dinner, chicken, pasta, 30-m...",...,0.67,0.83,10.54,20-30,4.54,1491.0,12,D,"[good give, perfect taste, fresh colour, nice ...",[stuck recipe]
2,"Yes, Virginia There is a Great Meatloaf",Absolutely delicious meatloaf and sauce! Those...,4,54257.0,232.0,80.0,"[ground beef, bread, egg, vidalia onions, tabl...",[1 1/2 lbs ground beef (ground shoulder roast ...,[1. Meatloaf: Combine meat loaf ingredients an...,"[low-carb, dinner, southern, time-to-make, cou...",...,0.17,8.15,15.30,80-90,4.21,1325.0,11,D,"[fond meatloaf, much thank, nice meatloaf, del...",[]
3,Jo Mama's World Famous Spaghetti,My kids will give up a steak dinner for this s...,10,22782.0,285.0,80.0,"[italian sausage, onion, garlic cloves, diced ...","[2 lbs Italian sausage, casings removed (mild ...","[1. In large, heavy stockpot, brown Italian sa...","[italian, pasta, dinner, weeknight, time-to-ma...",...,1.33,4.00,10.46,80-90,4.42,1273.0,16,D,"[good ingredient, great keeper, best sauce, ni...","[sausage drained, lower fat content]"
4,Pancakes,"This is really a great recipe! It is fast, sim...",1,25690.0,407.0,10.0,"[egg, milk, butter, flour, sugar, baking powde...","[1 egg, 3/4 cup milk, 2 tablespoons butter or ...","[1. Beat egg until fluffy., 2. Add milk and me...","[breakfast, 15-minutes-or-less, time-to-make, ...",...,0.10,0.37,0.69,10-20,4.43,1076.0,7,A,"[delicious easy recipe, simple delicious daugh...",[]


Inicializar la sesión de spark

In [ ]:
from pyspark.sql import SparkSession
sc = SparkContext
# Iniciar SparkSession
spark = SparkSession.builder.appName("SistemaRecomendacionRecetas").getOrCreate()

24/11/21 19:06:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Cargar los datos pickle usando pandas y convertirlos a DataFrames de PySpark

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd

# Crear la sesión de Spark y configurar el acceso a GCS
spark = SparkSession.builder \
    .appName("MiAplicacion") \
    .config("spark.jars", "gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "/path/to/your-service-account-file.json") \
    .getOrCreate()

# Leer los archivos pickle desde el bucket de GCS
bucket_path = "gs://mi-bucket-tfg/"

# Leer los archivos pickle con pandas
recipes_pd = pd.read_pickle(bucket_path + "datos/df_recipes_filtered.pkl")
reviews_pd = pd.read_pickle(bucket_path + "datos/df_reviews_filtered.pkl")

24/11/21 19:06:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
# Guardar los DataFrames de pandas en formato Parquet o CSV
recipes_pd.to_parquet("gs://mi-bucket-tfg/datos/recipes.parquet")
reviews_pd.to_parquet("gs://mi-bucket-tfg/datos/reviews.parquet")

# Leer los archivos Parquet con PySpark
recipes = spark.read.parquet("gs://mi-bucket-tfg/datos/recipes.parquet")
reviews = spark.read.parquet("gs://mi-bucket-tfg/datos/reviews.parquet")

*   **Dataset: Embeddings**

In [ ]:
# Configura el nombre de tu bucket y archivo
bucket_name = 'mi-bucket-tfg'
file_path = 'datos/df_recipes_embeddings.pkl'
local_path = '/tmp/df_recipes_embeddings.pkl'

# Conectar a Google Cloud Storage
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename(local_path)

# Cargar el archivo Pickle en un DataFrame
df_embeddings = pd.read_pickle(local_path)
df_embeddings.head()

,name,description,servings,recipe_id,recipe_grams,recipe_minutes,ingredients_list,ingredients_and_quantities,instructions,keywords,...,WR,normalized_name,recipe_name_embeddings,similarity,normalized_ingredients_list,ingredients_embeddings,normalized_keywords,keywords_embeddings,normalized_highlighted_aspects,aspects_embeddings
30,Roasted Asparagus,"I got this recipe from ""Simple Vegetarian Plea...",4,50847.0,119.0,25.0,"[asparagus, olive oil, kosher salt]","[1 lb asparagus, 1 1/2 tablespoons olive oil, ...","[1. Preheat oven to 425°F., 2. Cut off the woo...","[side, vegan, dinner, vegetarian, low-calorie,...",...,4.559188,roasted asparagus,"[0.70774525, 0.17281596, -0.53876466, 0.269132...",0.399164,"[asparagus, olive oil, kosher salt]","[0.068096995, -0.039195217, -0.66528445, -0.07...","[side, vegan, dinner, vegetarian, low calorie,...","[0.30943736, 0.32184526, 0.10604355, 0.0815918...","[delicious quick easy unbeatable combination, ...","[-0.25535214, -0.056583103, -0.012598676, -0.0..."
108,Mexican Stack-Up #RSC,"Ready, Set, Cook! Reynolds Wrap Contest Entry....",6,486261.0,351.0,75.0,"[poblano pepper, reynolds wrap foil, olive oil...","[1 poblano pepper, Reynolds Wrap Foil, 2 table...",[1. Preheat broiler to high broil. Set the ra...,"[mexican, low-carb, dinner, time-to-make, cour...",...,4.531710,mexican stackup rsc,"[-0.7105343, -0.46382508, -0.65078366, 0.13243...",0.581127,"[poblano pepper, reynolds wrap foil, olive oil...","[-0.6319319, -0.24501374, -0.21909977, 0.15086...","[mexican, low carb, dinner, time to make, cour...","[0.29413253, 0.050493628, -0.22796434, 0.00538...","[deliciousness gobbled, easy make, wonderful s...","[-0.2678564, -0.22381152, 0.007309506, -0.0815..."
46,Roasted Cauliflower &amp; 16 Roasted Cloves of...,"I find it totally amazing, that children and m...",6,106251.0,158.0,40.0,"[cauliflower, garlic cloves, fresh rosemary, s...",[1 large cauliflower (trimmed and cut into bit...,"[1. Mix oil, rosemary, salt, pepper and garlic...","[side, vegan, vegetarian, low-calorie, low-car...",...,4.515852,roasted cauliflower amp roasted cloves of garlic,"[0.12248431, -0.42801306, -0.38706228, -0.1775...",0.582425,"[cauliflower, garlic cloves, fresh rosemary, s...","[-0.2948644, -0.7789556, -0.25072205, -0.25720...","[side, vegan, vegetarian, low calorie, low car...","[0.27294207, 0.16783305, 0.15147586, 0.0267453...","[delicious rita, easy cooking, great hubby, fr...","[-0.5869685, -0.1143411, 0.2315941, -0.1252554..."
7,Kittencal's Moist Cheddar-Garlic Oven Fried Ch...,This is for serious garlic lovers. The chicken...,4,82102.0,185.0,60.0,"[butter, garlic, garlic powder, seasoning salt...","[1/3 cup butter, melted , 2 tablespoons minced...","[1. Preheat oven to 350°F., 2. Butter an 11 x ...","[chicken, dinner, 60-minutes-or-less, time-to-...",...,4.514544,kittencals moist cheddargarlic oven fried chic...,"[0.004746352, -0.12346557, -0.16879883, 0.1019...",0.412056,"[butter, garlic, garlic powder, seasoning salt...","[-0.5912476, 0.18889797, -0.24024855, -0.00016...","[chicken, dinner, 60 minutes or less, time to ...","[0.109381385, -0.22464582, 0.038818248, -0.292...","[great single cook, amazing chicken, full robu...","[-0.20479614, 0.11154066, -0.041402392, -0.242..."
38,Roasted Green Beans,"A great alternative to fast-food French fries,...",4,133821.0,154.0,25.0,"[green beans, olive oil, kosher salt, fresh gr...","[2 lbs green beans, 1 or 2 tablespoon olive o...","[1. Preheat oven to 400°F., 2. Wash, dry well,...","[lunch, side, vegan, vegetarian, lactose-free,...",...,4.514403,roasted green beans,"[-0.07211356, 0.03044193, -0.5078084, 0.067886...",0.602669,"[green beans, olive oil, kosher salt, fresh gr...","[-0.4692881, -0.05639726, -0.6351077, -0.14723...","[lunch, side, vegan, vegetarian, lactose free,...","[0.13365375, 0.11425532, -0.17680538, 0.114559...","[exciting way, nice roasted flavor, terrific s...","[-0.6486899, 0.41429377, -0.47726592, -0.00866..."


*   **Dataset: Reviews**

In [ ]:
from google.cloud import storage

# Configura el nombre de tu bucket y archivo
bucket_name = 'mi-bucket-tfg'
file_path = 'datos/df_reviews_filtered.pkl'
local_path = '/tmp/df_reviews_filtered.pkl'

# Conectar a Google Cloud Storage
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename(local_path)

# Cargar el archivo Pickle en un DataFrame
df_reviews = pd.read_pickle(local_path)
df_reviews.head()

,review_id,recipe_id,author_id,rating,review,review_date
0,9,4523,2046,2,i think i did something wrong because i could ...,2000
1,14,44,2085,5,An excellent dish.,2000
2,23,4684,2046,5,this is absolutely delicious. i even served i...,2000
3,28,1451,2416,1,Where does the bbq come in,2000
4,55,7762,2369,3,They taste better than they appear.\r\n,2000


# **3. Sistema de Recomendación de Filtrado Colaborativo**

## **Modelo ALS**
Este sistema de recomendación utiliza ALS (Alternating Least Squares), una técnica común en sistemas de recomendación basados en filtrado colaborativo. Este método descompone una matriz de interacciones (en este caso, calificaciones de recetas) en dos matrices más pequeñas: una para los usuarios y otra para los ítems, de manera que se minimice la diferencia entre las calificaciones reales y las predichas.

En este contexto, el modelo ALS genera un conjunto de recomendaciones personalizadas para cada usuario, donde cada usuario (author_id) y tiene una lista de recomendaciones. Cada recomendación incluye el recipe_id de la receta sugerida y su correspondiente predicted_rating, es decir, la puntuación predicha que refleja qué tan relevante será la receta para ese usuario.

Tuve que ejecutarlo en un notebook diferente ya que los anteriores utilizaban un entorno de python y este modelo de PySpark.

### **Paso 1: Preparación de los datos**

Hagamos un primer vistazo del esquema y de los datos luego de la transformacion a dataframes de PySpark:

In [ ]:
reviews.printSchema()

root
 |-- review_id: long (nullable = true)
 |-- recipe_id: long (nullable = true)
 |-- author_id: long (nullable = true)
 |-- rating: long (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: integer (nullable = true)



In [ ]:
reviews.show()

+---------+---------+---------+------+--------------------+-----------+
|review_id|recipe_id|author_id|rating|              review|review_date|
+---------+---------+---------+------+--------------------+-----------+
|        9|     4523|     2046|     2|i think i did som...|       2000|
|       14|       44|     2085|     5|  An excellent dish.|       2000|
|       23|     4684|     2046|     5|this is absolutel...|       2000|
|       28|     1451|     2416|     1|Where does the bb...|       2000|
|       55|     7762|     2369|     3|They taste better...|       2000|
|       59|     7205|     2178|     3|I think I cooked ...|       2000|
|       87|     8445|     3090|     5|I think this reci...|       2000|
|       89|     4366|     2178|     5|if you like oyste...|       2000|
|       99|     1857|     3241|     5|This greek chicke...|       2000|
|      128|     4296|     3438|     4|I found this meat...|       2000|
|      130|     4734|     3525|     5|I made this casse...|     

In [ ]:
recipes.printSchema()

root
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- servings: long (nullable = true)
 |-- recipe_id: double (nullable = true)
 |-- recipe_grams: double (nullable = true)
 |-- recipe_minutes: double (nullable = true)
 |-- ingredients_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ingredients_and_quantities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- instructions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- calories_100g: double (nullable = true)
 |-- fat_100g: double (nullable = true)
 |-- saturated_fat_100g: double (nullable = true)
 |-- sodium_100g: double (nullable = true)
 |-- carbohydrate_100g: double (nullable = true)
 |-- fiber_100g: double (nullable = true)
 |-- sugar_100g: double (nullable = true)
 |-- protein_100g: double (nullable = true)
 |-- time_grou

In [ ]:
recipes.show()

+--------------------+--------------------+--------+---------+------------+--------------+--------------------+--------------------------+--------------------+--------------------+-------------+--------+------------------+-----------+-----------------+----------+----------+------------+----------+----------+------------+-------------+----------+
|                name|         description|servings|recipe_id|recipe_grams|recipe_minutes|    ingredients_list|ingredients_and_quantities|        instructions|            keywords|calories_100g|fat_100g|saturated_fat_100g|sodium_100g|carbohydrate_100g|fiber_100g|sugar_100g|protein_100g|time_group|avg_rating|review_count|n_ingredients|nutriscore|
+--------------------+--------------------+--------+---------+------------+--------------+--------------------+--------------------------+--------------------+--------------------+-------------+--------+------------------+-----------+-----------------+----------+----------+------------+----------+------

Ajustamos los tipos de datos de las reseñas para asegurar que las columnas clave tengan los tipos correctos para su procesamiento posterior. Específicamente, convierte `author_id` y `recipe_id` a enteros (integer), ya que representan identificadores únicos, y convierte rating a un número flotante (float) para facilitar cálculos como promedios o predicciones.

In [ ]:
reviews = reviews.\
    withColumn('author_id', col('author_id').cast('integer')).\
    withColumn('recipe_id', col('recipe_id').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')


reviews.show()

+---------+---------+---------+------+--------------------+-----------+
|review_id|recipe_id|author_id|rating|              review|review_date|
+---------+---------+---------+------+--------------------+-----------+
|        9|     4523|     2046|   2.0|i think i did som...|       2000|
|       14|       44|     2085|   5.0|  An excellent dish.|       2000|
|       23|     4684|     2046|   5.0|this is absolutel...|       2000|
|       28|     1451|     2416|   1.0|Where does the bb...|       2000|
|       55|     7762|     2369|   3.0|They taste better...|       2000|
|       59|     7205|     2178|   3.0|I think I cooked ...|       2000|
|       87|     8445|     3090|   5.0|I think this reci...|       2000|
|       89|     4366|     2178|   5.0|if you like oyste...|       2000|
|       99|     1857|     3241|   5.0|This greek chicke...|       2000|
|      128|     4296|     3438|   4.0|I found this meat...|       2000|
|      130|     4734|     3525|   5.0|I made this casse...|     

Hacemos lo mismo con el `recipe_id` para aseguraron de que este en formato integer (entero).

In [ ]:
recipes = recipes.\
    withColumn('recipe_id', col('recipe_id').cast('integer'))

recipes.show()

+--------------------+--------------------+--------+---------+------------+--------------+--------------------+--------------------------+--------------------+--------------------+-------------+--------+------------------+-----------+-----------------+----------+----------+------------+----------+----------+------------+-------------+----------+
|                name|         description|servings|recipe_id|recipe_grams|recipe_minutes|    ingredients_list|ingredients_and_quantities|        instructions|            keywords|calories_100g|fat_100g|saturated_fat_100g|sodium_100g|carbohydrate_100g|fiber_100g|sugar_100g|protein_100g|time_group|avg_rating|review_count|n_ingredients|nutriscore|
+--------------------+--------------------+--------+---------+------------+--------------+--------------------+--------------------------+--------------------+--------------------+-------------+--------+------------------+-----------+-----------------+----------+----------+------------+----------+------

Unimos ambos dataframes:

In [ ]:
reviews_with_recipes = reviews.join(recipes, ['recipe_id'], 'left')

reviews_with_recipes.show()

24/11/21 19:07:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+---------+---------+------+--------------------+-----------+--------------------+--------------------+--------+------------+--------------+--------------------+--------------------------+--------------------+--------------------+-------------+--------+------------------+-----------+-----------------+----------+----------+------------+----------+----------+------------+-------------+----------+
|recipe_id|review_id|author_id|rating|              review|review_date|                name|         description|servings|recipe_grams|recipe_minutes|    ingredients_list|ingredients_and_quantities|        instructions|            keywords|calories_100g|fat_100g|saturated_fat_100g|sodium_100g|carbohydrate_100g|fiber_100g|sugar_100g|protein_100g|time_group|avg_rating|review_count|n_ingredients|nutriscore|
+---------+---------+---------+------+--------------------+-----------+--------------------+--------------------+--------+------------+--------------+--------------------+-------------

**Calculo de parisity**

Determinamos la dispersión de la matriz de utilidad, lo que nos permite conocer el porcentaje de celdas vacías en la matriz. Este análisis es relevante porque:

* En aplicaciones reales, los sistemas de recomendación trabajan con matrices altamente dispersas.
* Comprender el nivel de dispersión facilita la detección de retos, como la falta de historial de interacciones para nuevos usuarios o recetas.

In [ ]:
# Calcular el numerador: número total de calificaciones
numerator = reviews.select("rating").count()

# Calcular el número de usuarios distintos (author_id) y recetas distintas (recipe_id)
num_users = reviews.select("author_id").distinct().count()
num_recipes = reviews.select("recipe_id").distinct().count()

# Calcular el denominador: número total de combinaciones posibles entre usuarios y recetas
denominator = num_users * num_recipes

# Calcular la dispersión (sparsity)
sparsity = (1.0 - (numerator * 1.0) / denominator) * 100
print("El DataFrame de reseñas está ", "%.2f" % sparsity + "% vacío.")

El DataFrame de reseñas está  99.99% vacío.


El resultado  "99.99% vacío", significa que la matriz de utilidad que representa las interacciones entre usuarios y recetas es extremadamente dispersa.

Analizamos la cantidad de reseñas por usuario (`author_id`) y por receta (`recipe_id`) para:
* Identificar usuarios activos que han calificado muchas recetas.
* Detectar recetas populares que han recibido muchas calificaciones.

In [ ]:
# Cantidad de reseñas por autor
author_id_ratings = reviews.groupBy("author_id").count().orderBy('count', ascending=False)
author_id_ratings.show()

+---------+-----+
|author_id|count|
+---------+-----+
|    37449| 2041|
|   424680| 1680|
|   199848| 1621|
|   383346| 1613|
|   226863| 1543|
|   305531| 1528|
|   133174| 1477|
|     4470| 1473|
|   169430| 1464|
|   128473| 1385|
|   369715| 1302|
|    89831| 1202|
|   286566| 1186|
|    58104| 1090|
|   498271| 1070|
|    95743| 1058|
|   140132| 1028|
|   280271| 1018|
|    88099|  959|
|     5060|  959|
+---------+-----+
only showing top 20 rows



Hay un solo autor con mas de 2.000 reseñas, y solo 5 arriba de 1.500 reseñas.

In [ ]:
# Cantidad de reseñas por receta
recipe_id_ratings = reviews.groupBy("recipe_id").count().orderBy('count', ascending=False)
recipe_id_ratings.show()

+---------+-----+
|recipe_id|count|
+---------+-----+
|    45809| 2892|
|    39087| 1491|
|    54257| 1325|
|    22782| 1273|
|    25690| 1076|
|    68955|  910|
|    33919|  885|
|    82102|  862|
|    85629|  805|
|    99476|  785|
|    26110|  782|
|   129926|  775|
|    10744|  739|
|   150863|  728|
|    63689|  726|
|    38298|  682|
|    65768|  660|
|    32614|  656|
|   117892|  614|
|    87782|  600|
+---------+-----+
only showing top 20 rows



Solo hay 5 recetas con mas de mil reseñas, de las cuales una sola tiene arriba de las 2mil.

### **Paso 2: Construcción y entrenamiento del modelo ALS**

Creamos los conjuntos de prueba y de entrenamiento.

In [ ]:
(train, test) = reviews.randomSplit([0.8, 0.2], seed=44)

Ajustamos el modelo ALS con un conjunto fijo de hiperparámetros. Como mejora futura se podría implementar un CrossValidation para mejorar el rendimiento del modelo.

In [ ]:
als = ALS(
    userCol="author_id",
    itemCol="recipe_id",
    ratingCol="rating",
    rank=50,                      # Número de factores latentes
    regParam=0.1,                 # Parámetro de regularización
    nonnegative=True,             # Asegurar que las predicciones sean positivas
    implicitPrefs=False,          # Preferencias explícitas (calificaciones)
    coldStartStrategy="drop"      # Evitar predicciones nulas
)

Entrenamos el modelo con el conjunto de entrenamiento

In [ ]:
model = als.fit(train)

### **Paso 3: Definición de la metrica a evaluar**

Utilizamos el evaluador RegressionEvaluator con la métrica RMSE (Root Mean Square Error) para medir la diferencia entre las calificaciones reales (rating) y las predicciones generadas (prediction). Evaluamos el modelo en el conjunto de prueba:

In [ ]:
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

In [ ]:
test_predictions = model.transform(test)
rmse = evaluator.evaluate(test_predictions)

print(f"RMSE en el conjunto de prueba: {rmse}")

RMSE en el conjunto de prueba: 1.165438772617933


Como las calificaciones (rating) están en un rango de 1 a 5, un RMSE de 1.16 significa que el error promedio es de aproximadamente una estrella en el sistema de calificaciones. Esto podría ser aceptable para un sistema inicial, pero un RMSE más cercano a 0.5 o menos sería ideal en un modelo futuro y mejorado.

### **Paso 4: Generación de las recomendaciones**
Generamos las recomendaciones para todos los usuarios (5 para cada uno).

In [ ]:
nrecommendations = model.recommendForAllUsers(5)
nrecommendations.limit(5).show()

24/11/21 19:15:34 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


+---------+--------------------+
|author_id|     recommendations|
+---------+--------------------+
|     1533|[{327537, 6.25278...|
|     1634|[{145206, 4.97983...|
|     1938|[{334220, 0.0}, {...|
|     2059|[{327537, 5.56453...|
|     2148|[{327537, 5.55211...|
+---------+--------------------+



### **Paso 5: Impresión de las recomendaciones**

Volvemos a definir la función para imprimir las recetas.

In [ ]:
def imprimir_recetas(df):
    for index, row in df.iterrows():
        print(f"\n{row['name']}")
        print(f"↳ {row['description']}")
        print(f"\nServings: {row['servings']}")
        print(f"Recipe Grams: {row['recipe_grams']}g")
        print(f"Preparation Time: {int(row['recipe_minutes'])} minutes")
        print(f"Nutriscore: {row['nutriscore']}")

        # Imprimir los ingredientes
        print(f"\nIngredients and Quantities:")
        for ingredient in row['ingredients_and_quantities']:
            print(f"  - {ingredient}")

        # Imprimir las instrucciones
        print(f"\nInstructions:")
        for instruction in row['instructions']:
            print(f"{instruction}")

        print("\n" + "-"*100)

La función `show_user_recommendations` recibe un ID de usuario, valida que sea un número entero y, si es válido, filtra el DataFrame `nrecommendations` para obtener las recomendaciones asociadas a ese usuario. Extrae los valores de `recipe_id` y `predicted_rating`, los convierte a un DataFrame de Pandas y utiliza estos IDs para filtrar `df_recipes` y obtener las recetas recomendadas. Finalmente, muestra los detalles de las recetas utilizando la función `imprimir_recetas`.

In [ ]:
def show_user_recommendations(user_id, nrecommendations, df_recipes):
    try:
        # Convert the user ID to integer (if necessary)
        user_id = int(user_id)
    except ValueError:
        print("The entered ID is invalid. Please enter an integer.")
        return

    # Filter recommendations for the given user ID
    user_recommendations = nrecommendations.filter(f'author_id = {user_id}')

    # Check if there are any recommendations for this user
    if user_recommendations.count() > 0:
        print(f"Recommendations for user ID {user_id}:")

        # Explode recommendations into individual rows
        exploded_recommendations = user_recommendations.select(
            col("author_id"),
            explode(col("recommendations")).alias("recommendation")
        )

        # Extract recipe_id and predicted_rating
        extracted_recommendations = exploded_recommendations.select(
            col("recommendation.recipe_id").alias("recipe_id"),
            col("recommendation.rating").alias("predicted_rating")
        )

        recomendaciones_pandas = extracted_recommendations.toPandas()

        recipe_ids = recomendaciones_pandas["recipe_id"].tolist()
        recetas_filtradas = df_recipes[df_recipes["recipe_id"].isin(recipe_ids)]

        imprimir_recetas(recetas_filtradas)
    else:
        print(f"No recommendations were found for user ID {user_id}.")

In [ ]:
'''user_id = input("Please enter your user ID: ")
show_user_recommendations(user_id, nrecommendations, df_recipes)'''

'user_id = input("Please enter your user ID: ")\nshow_user_recommendations(user_id, nrecommendations, df_recipes)'

In [ ]:
user_id = 1533
show_user_recommendations(user_id, nrecommendations, df_recipes)

Recommendations for user ID 1533:



Tasty Vegetarian Chili
↳ Chilli has always been my fave dish and i have tried many variations on the basic content.  This is the best I've come up with.  The ingredients can be varied personal taste, the final two ingredients are what makes this dish stand out!

Servings: 4
Recipe Grams: 517.0g
Preparation Time: 80 minutes
Nutriscore: A

Ingredients and Quantities:
  - 3 tablespoons vegetable oil
  - 2 onions, coursely chopped 
  - 2 garlic cloves, crushed 
  - 1 large green pepper, coursely chopped 
  - 1/2 lb mushroom, sliced 
  - 2  or 4 sticks celery, sliced 
  - 1 (14 ounce) can chopped tomatoes
  - 2 (14 ounce) cans red kidney beans, drained 
  - 1/2 or 1 cup Bulgar wheat
  - 2 tablespoons tomato puree
  - 1/2 or 1 pint vegetable stock (instant bouillion works well)
  - 1 teaspoon oregano
  - 1 teaspoon chili powder
  - 1 tablespoon ground cumin
  - 1 teaspoon ground coriander
  - 1  or 2 teaspoon mint
  - 1/2 teaspoon basil
  - 2 teaspoons English mustard
  - 1  or 2 tablespoon

Para poder utilizar las recomendaciones en el MVP, vamos a exportar las recomendaciones a un csv.

In [ ]:
from pyspark.sql import functions as F

# Explode the recommendations to break them into separate rows
exploded_recommendations = nrecommendations.select(
    F.col("author_id"),
    F.explode(F.col("recommendations")).alias("recommendation")
)

# Extract recipe_id and predicted_rating
extracted_recommendations = exploded_recommendations.select(
    F.col("author_id"),
    F.col("recommendation.recipe_id").alias("recipe_id"),
    F.col("recommendation.rating").alias("predicted_rating")
)

# Convert to Pandas DataFrame
recommendations_pandas = extracted_recommendations.toPandas()

# Group by author_id and aggregate the recommendations and ratings into lists
grouped_recommendations = recommendations_pandas.groupby('author_id').agg({
    'recipe_id': lambda x: x.tolist(),
    'predicted_rating': lambda x: x.tolist()
}).reset_index()

# Define local and bucket paths
csv_file_path = 'all_recommendations.csv'
bucket_name = 'mi-bucket-tfg'
destination_blob_path = 'datos/all_recommendations.csv'

# Save the grouped recommendations to a local CSV file
grouped_recommendations.to_csv(csv_file_path, index=False)

# Connect to the Google Cloud Storage client
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(destination_blob_path)

# Upload the CSV file to the GCP bucket
blob.upload_from_filename(csv_file_path)

print(f"The CSV file has been successfully uploaded to '{bucket_name}/{destination_blob_path}'")

The CSV file has been successfully uploaded to 'mi-bucket-tfg/datos/all_recommendations.csv'


## **Modelo SVD**
El SVD (Singular Value Decomposition) es una técnica de descomposición matricial utilizada en sistemas de recomendación colaborativos para identificar patrones latentes en los datos de interacción usuario-producto, reducir dimensionalidad y predecir valores faltantes. Esto mejora las recomendaciones al capturar relaciones implícitas entre usuarios y productos, incluso con datos dispersos. 

### **Paso 1: Crear una matriz de interacciones entre usuarios y recetas**
Para preparar los datos en un formato adecuado para el análisis, transformamos la información asignando un identificador numérico único a cada usuario (`user_idx`) y a cada receta (`recipe_idx`).

Esta conversión es esencial porque muchos algoritmos de recomendación, como los basados en factorización de matrices, están diseñados para trabajar con datos numéricos en lugar de identificadores alfanuméricos. A pesar de que nuestros datos ya eran numericos, no eran consecutivos. Asignar índices consecutivos optimiza tanto la memoria como la compatibilidad con los algoritmos de recomendación, es por ello que vamos a rasignar los indices.

In [ ]:
user_map = {user: i for i, user in enumerate(df_reviews['author_id'].unique())}
recipe_map = {recipe: i for i, recipe in enumerate(df_reviews['recipe_id'].unique())}


df_reviews['user_idx'] = df_reviews['author_id'].map(user_map)
df_reviews['recipe_idx'] = df_reviews['recipe_id'].map(recipe_map)

Se crea una matriz donde:
* Filas representan a los usuarios.
* Columnas representan a las recetas.
* Valores contienen las calificaciones dadas por los usuarios a las recetas (puntaje de 0 a 5).

Esta matriz es conocida como matriz de interacciones y permite representar de manera estructurada las relaciones entre usuarios y recetas. En la práctica, la matriz suele ser dispersa (sparse) porque la mayoría de los usuarios no han calificado todas las recetas, lo que genera muchas celdas vacías (sin datos).

In [ ]:
interaction_matrix = csr_matrix(
    (df_reviews['rating'], (df_reviews['user_idx'], df_reviews['recipe_idx'])),
    shape=(len(user_map), len(recipe_map))
)

### **2. Centrar las calificaciones**
Para ajustar el modelo y reducir los sesgos individuales, se normalizan las calificaciones centrándolas respecto al promedio de cada usuario. Este proceso es clave para manejar diferencias en los estilos de calificación: algunos usuarios califican de manera estricta (con puntuaciones bajas) y otros son más generosos (con puntuaciones altas).

El enfoque consiste en:
* **Calcular el promedio de calificaciones por usuario:** Esto captura la tendencia de cada usuario.
* **Restar el promedio individual a cada calificación**

De esta forma, se eliminan los sesgos y se enfocan las diferencias relativas respecto a las preferencias individuales.

In [ ]:
# Calcular la media de calificaciones por usuario
user_means = df_reviews.groupby('author_id')['rating'].mean()

# Calcular las calificaciones centradas
df_reviews['centered_rating'] = df_reviews.apply(
    lambda row: row['rating'] - user_means[row['author_id']], axis=1
)

Al igual que en la matriz inicial, se construye una matriz dispersa (sparse matrix), pero esta vez utilizando las calificaciones centradas:

In [ ]:
# Crear matriz dispersa con calificaciones centradas
centered_interaction_matrix = csr_matrix(
    (df_reviews['centered_rating'], (df_reviews['user_idx'], df_reviews['recipe_idx'])),
    shape=(len(user_map), len(recipe_map))
)

### **Paso 3: Factorización de matrices con SVD**

La factorización de matrices mediante SVD (Singular Value Decomposition) se utiliza para descomponer la matriz de interacciones centrada en factores latentes. Esto permite descubrir relaciones complejas entre usuarios y recetas que no son evidentes a simple vista.

* `user_factors`: Representa las preferencias de los usuarios en dimensiones abstractas.
* `item_factors`: Representa las características de las recetas en las mismas dimensiones.

In [ ]:
# Configurar y aplicar SVD
n_components = 100
svd = TruncatedSVD(n_components=n_components, random_state=42)
svd.fit(centered_interaction_matrix)

TruncatedSVD(n_components=100, random_state=42)

Los factores latentes (`user_factors` e `item_factors`) se calculan a partir de la matriz original. Estos factores se usarán para predecir calificaciones y encontrar similitudes entre usuarios o recetas.

In [ ]:
# Obtener factores de usuario y receta
user_factors = svd.transform(centered_interaction_matrix)
item_factors = svd.components_.T

### **Paso 4: Predicción de calificaciones**
Una vez obtenidos los factores latentes de los usuarios y las recetas, se pueden predecir las calificaciones faltantes mediante la multiplicación de las matrices de factores. Este proceso es clave para hacer recomendaciones personalizadas.

La predicción de calificaciones centradas se obtiene mediante el producto punto (dot product) de los factores de usuario y los factores de receta transpuestos.

**Revertir la normalización (añadir las medias):** Como las calificaciones fueron centradas, se necesita revertir este paso para obtener las calificaciones predichas en su escala original. Para hacerlo, se suma el promedio de calificaciones de cada usuario a las predicciones centradas, recuperando el sesgo individual de cada usuario.

In [ ]:
# Predicción de calificaciones centradas
predicted_ratings = np.dot(user_factors, item_factors.T)

# Revertir la normalización (añadir las medias)
predicted_ratings_full = predicted_ratings + user_means.values.reshape(-1, 1)

### **Paso 5: Evaluación del modelo**
Se compara la matriz de calificaciones reales con las predichas.

El RMSE (Root Mean Squared Error) mide la precisión del modelo. Un RMSE bajo significa que las predicciones están más cerca de las calificaciones reales.

In [ ]:
# Recuperar calificaciones reales y predichas
test_data = df_reviews[['user_idx', 'recipe_idx', 'rating']]
test_predicted = predicted_ratings[test_data['user_idx'], test_data['recipe_idx']]

# Calcular RMSE
rmse = np.sqrt(mean_squared_error(test_data['rating'], test_predicted))
print(f'RMSE: {rmse}')

RMSE: 4.595339250161219


### **Paso 6: Optimización de hiperparámetros**
Se prueban diferentes valores para el número de dimensiones latentes (n_components) para encontrar el mejor balance entre complejidad y precisión.

El RMSE se calcula para cada configuración, y se guarda el mejor resultado.

In [ ]:
best_rmse = float('inf')
best_n_components = None

components_range = [50, 100, 150, 200]

for n_components in components_range:
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    svd.fit(centered_interaction_matrix)
    user_factors = svd.transform(centered_interaction_matrix)
    item_factors = svd.components_.T

    predicted_ratings = np.dot(user_factors, item_factors.T)
    predicted_ratings_full = predicted_ratings + user_means.values.reshape(-1, 1)

    # Calcular RMSE
    test_predicted = predicted_ratings[test_data['user_idx'], test_data['recipe_idx']]
    rmse = np.sqrt(mean_squared_error(test_data['rating'], test_predicted))

    if rmse < best_rmse:
        best_rmse = rmse
        best_n_components = n_components

print(f'Mejor número de componentes: {best_n_components} con RMSE: {best_rmse}')

Mejor número de componentes: 200 con RMSE: 4.58817047534678


Un RMSE de 4.58 significa que, en promedio, las calificaciones predichas por el modelo están equivocadas por 4.58 puntos con respecto a las calificaciones reales.
Dado que el rango es de 0 a 5, el RMSE obtenido es un valor extremadamente grande en relación con el rango disponible, lo que indica que el modelo no está haciendo buenas predicciones.

## **Alternativa con Surprise**

Surprise es una biblioteca de Python diseñada para construir y evaluar sistemas de recomendación, especialmente aquellos basados en técnicas colaborativas. Ofrece herramientas para manejar datos de calificaciones, implementar algoritmos de recomendación (como SVD, KNN, etc.), y evaluar su desempeño mediante métricas como RMSE o MAE. Es fácil de usar y permite experimentar con modelos para encontrar el más adecuado.

El código convierte un DataFrame de pandas en un formato compatible con Surprise. Define el rango de calificaciones de 0 a 5 que contiene el detaset.

In [ ]:
# Crear dataset compatible con Surprise
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df_reviews[['author_id', 'recipe_id', 'rating']], reader)

Se usa `GridSearchCV` para probar diferentes combinaciones de hiperparámetros y encontrar el mejor modelo.

In [ ]:
# Hiperparámetros y validación cruzada
param_grid = {
    'n_factors': [100, 150],
    'n_epochs': [30, 40],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.1]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)

1. Se código entrena y evalúa el mejor modelo encontrado en una búsqueda de parámetros con Surprise.
2. Se recupera el modelo óptimo basado en el menor RMSE (`gs.best_estimator['rmse']`).
3. Se divide el dataset en entrenamiento (80%) y prueba (20%)
4. Se entrena el modelo con el conjunto de entrenamiento. 
5. Se evalúa en el conjunto de prueba y se calcula el RMSE de las predicciones para medir su precisión, mostrando el resultado final.

In [ ]:
# Evaluar el mejor modelo
best_estimator = gs.best_estimator['rmse']
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
best_estimator.fit(trainset)
predictions = best_estimator.test(testset)
rmse = accuracy.rmse(predictions)
print(f"Mejor RMSE con Surprise: {rmse}")

RMSE: 1.1579
Mejor RMSE con Surprise: 1.1578725990701522


Dado que el RMSE del modelo ALS fue de 1.165 y el obtenido en SVD fue de 1.157, las diferencias entre el rendimiento de ambos modelos no es significativa. Sin embargo, dado que SVD requirio 208 GB de RAM, mientras que ALS funciono a la perfección con 15 GB, optaremos por utilizar este ultimo ya que resulto mas eficiente computaciónalmente.